In [1]:
import pandas as pd

In [2]:
def primera_llamada(name):

    df = pd.read_csv('ibex35.csv')

    name = df[df['empresa'] == name]

    name = name [['date','open','high','low','close','adj close']]

    name['date'] = pd.to_datetime(name['date'], format="%Y-%m-%d")

    name.sort_values(by='date',ascending= True, inplace=True)

    fecha_inicio = '2021-01-01'
    fecha_fin = '2024-03-01'

    name = pd.DataFrame(name[(name['date'] >= fecha_inicio) & (name['date'] <= fecha_fin)]).reset_index(drop=True)

    return name

In [3]:
indra = primera_llamada('Indra')

indra

,date,open,high,low,close,adj close
0,2021-01-04,7.115000,7.115000,6.810000,6.850000,6.485601
1,2021-01-05,6.800000,6.965000,6.785000,6.965000,6.594484
2,2021-01-06,7.030000,7.095000,6.915000,7.065000,6.689164
3,2021-01-07,7.090000,7.090000,6.835000,6.940000,6.570814
4,2021-01-08,6.980000,7.075000,6.965000,7.060000,6.684429
...,...,...,...,...,...,...
807,2024-02-26,15.980000,16.180000,15.980000,16.070000,16.070000
808,2024-02-27,16.200001,16.240000,15.920000,16.030001,16.030001
809,2024-02-28,16.500000,17.590000,16.440001,17.080000,17.080000
810,2024-02-29,17.299999,17.959999,17.190001,17.670000,17.670000


In [4]:

def preparar_modelo(DF):


    DF_filt = DF[['date','close']]

    DF_filt.close = DF.close.round(2)

    DF_filt= DF_filt.reset_index(drop=True)

    new_df = pd.DataFrame(columns=['d-4', 'd-3', 'd-2', 'd-1', 'do'])

    for i in range(len(new_df.columns)):
        new_df[new_df.columns[i]] = DF_filt['close'].shift(-i)

    new_df['date']= DF_filt.date

    new_df.insert(0, 'date', new_df.pop('date'))

    new_df.dropna(axis=0,inplace=True)

    return new_df



In [5]:
new_df = preparar_modelo(indra)

new_df

C:\Users\javie\AppData\Local\Temp\ipykernel_31956\4040868792.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF_filt.close = DF.close.round(2)


,date,d-4,d-3,d-2,d-1,do
0,2021-01-04,6.85,6.96,7.06,6.94,7.06
1,2021-01-05,6.96,7.06,6.94,7.06,7.10
2,2021-01-06,7.06,6.94,7.06,7.10,7.19
3,2021-01-07,6.94,7.06,7.10,7.19,7.43
4,2021-01-08,7.06,7.10,7.19,7.43,7.48
...,...,...,...,...,...,...
803,2024-02-20,15.83,15.75,15.89,15.93,16.07
804,2024-02-21,15.75,15.89,15.93,16.07,16.03
805,2024-02-22,15.89,15.93,16.07,16.03,17.08
806,2024-02-23,15.93,16.07,16.03,17.08,17.67


In [6]:

def modelo_train(new_df):

    # Importar herramientas:

    from sklearn.model_selection import train_test_split as tts 
    from sklearn.ensemble import RandomForestRegressor as RFR 
    from sklearn.linear_model import OrthogonalMatchingPursuit as OMP
    from sklearn.metrics import mean_squared_error as mse

    # Separar X e y:

    X = new_df.drop(['do','date'],axis=1)
    y = new_df.do

    # Ejecutar tts:

    X_train, X_test, y_train, y_test = tts(X, y, train_size=0.8, test_size=0.2, random_state=42)
    X_train.shape, X_test.shape, y_train.shape, y_test.shape

    # Iniciar Modelos

    rfr=RFR()
    omp= OMP()

    # Entrenar

    rfr.fit(X_train, y_train)
    omp.fit(X_train, y_train)
     
    # Predecir

    y_pred_rfr=rfr.predict(X_test)
    y_pred_omp=omp.predict(X_test)

    # Mostrar rmse de cada modelo   
   
    return 'RMSE RFR ======',mse(y_test, y_pred_rfr, squared=False),'REMS OMP ======',mse(y_test, y_pred_omp, squared=False)

In [7]:
modelo_train(new_df)

('RMSE RFR ======', 0.191193687909136, 'REMS OMP ======', 0.1621130282205618)

In [8]:
def model(new_df):

    fecha_inicio = '2024-01-01'
    fecha_fin = '2024-02-26'

    # Filtrar el DataFrame
    mask = (new_df['date'] >= fecha_inicio) & (new_df['date'] <= fecha_fin)
    df_pred = new_df.loc[mask]

    df_pred_1 = df_pred[['d-4','d-3','d-2','d-1']]
    
    
    from sklearn.linear_model import OrthogonalMatchingPursuit as OMP

    omp= OMP()

    predicciones_y = []

    # Iterar sobre las 40 posiciones
    for i in range(40):
        # Seleccionar la porción correspondiente de X e y para esta iteración
        X_iteracion = new_df.drop(['date', 'do'], axis=1)[:-40+i]
        y_iteracion = new_df.do[:-40+i]

        # Ajustar el modelo
        omp.fit(X_iteracion, y_iteracion)

        # Seleccionar la fila correspondiente de df_pred para la predicción
        pred = df_pred_1[i:i+1]

        # Realizar la predicción y almacenarla en la lista
        y_pred_omp = omp.predict(pred)
        predicciones_y.append(y_pred_omp[0])   

    pred = pd.DataFrame(predicciones_y)

    pred.columns = ['prediccion']

    df_pred = df_pred.reset_index(drop=True)
    
    df_pred['pred'] = pred.prediccion.round(2)

    df_pred['error'] = df_pred.pred.round(2) - df_pred.do  

    return  df_pred

In [9]:
df_pred = model(new_df)

df_pred

,date,d-4,d-3,d-2,d-1,do,pred,error
0,2024-01-02,14.08,13.93,13.97,14.01,14.17,14.01,-0.16
1,2024-01-03,13.93,13.97,14.01,14.17,14.07,14.17,0.10
2,2024-01-04,13.97,14.01,14.17,14.07,14.26,14.07,-0.19
3,2024-01-05,14.01,14.17,14.07,14.26,14.88,14.26,-0.62
4,2024-01-08,14.17,14.07,14.26,14.88,15.21,14.88,-0.33
5,2024-01-09,14.07,14.26,14.88,15.21,15.31,15.21,-0.10
6,2024-01-10,14.26,14.88,15.21,15.31,15.24,15.31,0.07
7,2024-01-11,14.88,15.21,15.31,15.24,15.65,15.24,-0.41
8,2024-01-12,15.21,15.31,15.24,15.65,15.80,15.65,-0.15
9,2024-01-15,15.31,15.24,15.65,15.80,15.67,15.81,0.14


In [10]:
def escenario(x):

    import math

    df_pred['stock'] = (x/df_pred.do).apply(math.floor)

    df_pred['day_gap'] = df_pred.do.diff()

    df_pred['money'] = df_pred.stock*df_pred.day_gap

    df_pred['evolution'] = x + df_pred['money'].iloc[1:].cumsum().round(2)

    max_money_index = df_pred['money'].idxmax()
    
    df_pred['opti_real'] = x + df_pred['money'].iloc[max_money_index:].cumsum().round(2)

    min_money_index = df_pred['pred'].idxmin()
    
    df_pred['opti_pred'] = x + df_pred['money'].iloc[min_money_index:].cumsum().round(2)

    return df_pred

In [11]:
df_pred = escenario(100)

df_pred

,date,d-4,d-3,d-2,d-1,do,pred,error,stock,day_gap,money,evolution,opti_real,opti_pred
0,2024-01-02,14.08,13.93,13.97,14.01,14.17,14.01,-0.16,7,NaN,NaN,NaN,NaN,NaN
1,2024-01-03,13.93,13.97,14.01,14.17,14.07,14.17,0.10,7,-0.10,-0.70,99.30,NaN,99.30
2,2024-01-04,13.97,14.01,14.17,14.07,14.26,14.07,-0.19,7,0.19,1.33,100.63,NaN,100.63
3,2024-01-05,14.01,14.17,14.07,14.26,14.88,14.26,-0.62,6,0.62,3.72,104.35,NaN,104.35
4,2024-01-08,14.17,14.07,14.26,14.88,15.21,14.88,-0.33,6,0.33,1.98,106.33,NaN,106.33
5,2024-01-09,14.07,14.26,14.88,15.21,15.31,15.21,-0.10,6,0.10,0.60,106.93,NaN,106.93
6,2024-01-10,14.26,14.88,15.21,15.31,15.24,15.31,0.07,6,-0.07,-0.42,106.51,NaN,106.51
7,2024-01-11,14.88,15.21,15.31,15.24,15.65,15.24,-0.41,6,0.41,2.46,108.97,NaN,108.97
8,2024-01-12,15.21,15.31,15.24,15.65,15.80,15.65,-0.15,6,0.15,0.90,109.87,NaN,109.87
9,2024-01-15,15.31,15.24,15.65,15.80,15.67,15.81,0.14,6,-0.13,-0.78,109.09,NaN,109.09


In [12]:
def dafatrame(df_pred,name):
    

    df_pred['name'] = name
    
    df_pred = df_pred[['name','date','do','pred','error','stock','day_gap','money','evolution','opti_real','opti_pred']]

    return df_pred

In [13]:
df= dafatrame(df_pred,'indra')

df

,name,date,do,pred,error,stock,day_gap,money,evolution,opti_real,opti_pred
0,indra,2024-01-02,14.17,14.01,-0.16,7,NaN,NaN,NaN,NaN,NaN
1,indra,2024-01-03,14.07,14.17,0.10,7,-0.10,-0.70,99.30,NaN,99.30
2,indra,2024-01-04,14.26,14.07,-0.19,7,0.19,1.33,100.63,NaN,100.63
3,indra,2024-01-05,14.88,14.26,-0.62,6,0.62,3.72,104.35,NaN,104.35
4,indra,2024-01-08,15.21,14.88,-0.33,6,0.33,1.98,106.33,NaN,106.33
5,indra,2024-01-09,15.31,15.21,-0.10,6,0.10,0.60,106.93,NaN,106.93
6,indra,2024-01-10,15.24,15.31,0.07,6,-0.07,-0.42,106.51,NaN,106.51
7,indra,2024-01-11,15.65,15.24,-0.41,6,0.41,2.46,108.97,NaN,108.97
8,indra,2024-01-12,15.80,15.65,-0.15,6,0.15,0.90,109.87,NaN,109.87
9,indra,2024-01-15,15.67,15.81,0.14,6,-0.13,-0.78,109.09,NaN,109.09


In [14]:
df.to_csv('indra.csv',index=False)